In [1]:
!pip install kaggle

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105797 sha256=4f37b38c7cbeda8dd7ea5d9ad05e00bab7453a11b236cc600bf15e543497d132
  Stored in directory: c:\users\dai.studentsdc\appdata\local\pip\cache\wheels\46\d2\26\84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad938489983
Successfully built kaggle


In [3]:
!kaggle competitions download -c playground-series-s3e24


  0%|          | 0.00/6.81M [00:00<?, ?B/s]
 15%|#4        | 1.00M/6.81M [00:01<00:08, 695kB/s]
 29%|##9       | 2.00M/6.81M [00:01<00:03, 1.39MB/s]
 44%|####4     | 3.00M/6.81M [00:01<00:01, 2.23MB/s]
 73%|#######3  | 5.00M/6.81M [00:02<00:00, 4.20MB/s]
100%|##########| 6.81M/6.81M [00:02<00:00, 3.36MB/s]


In [17]:
from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, roc_auc_score
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
import os


In [10]:
os.chdir(r'playground-series-s3e24')

In [20]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
id,,,,,,,,,,,,,,,,,,,,,
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,57,126,16.2,1,1.1,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,...,45,93,17.4,1,0.8,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,...,38,102,15.9,1,1.0,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,...,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,40,155,45,69.0,1.5,2.0,1,1,127,80,...,72,159,14.5,1,0.8,25,26,13,0,0
159252,50,155,75,82.0,1.0,1.0,1,1,120,80,...,64,108,14.5,1,0.6,21,20,18,0,0
159253,40,160,50,66.0,1.5,1.0,1,1,114,70,...,87,93,10.9,1,0.6,15,9,12,0,0


In [21]:
X = train.drop(columns='smoking')
y = train.smoking

In [22]:
log = LogisticRegression(random_state=24)
svm = SVC(probability=True, random_state=24)
dtc = DecisionTreeClassifier(random_state=24)
bagg = BaggingClassifier(random_state=24)

In [23]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
params = {'estimator': [log, svm, dtc],
         'n_estimators' : [10,20,30,40,50]}
gcv = GridSearchCV(bagg, param_grid=params, cv= kfold, scoring='neg_log_loss', verbose=3)
gcv.fit(X,y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END estimator=LogisticRegression(random_state=24), n_estimators=10;, score=-0.524 total time=  11.6s
[CV 2/5] END estimator=LogisticRegression(random_state=24), n_estimators=10;, score=-0.518 total time=  11.3s
[CV 3/5] END estimator=LogisticRegression(random_state=24), n_estimators=10;, score=-0.520 total time=  11.0s
[CV 4/5] END estimator=LogisticRegression(random_state=24), n_estimators=10;, score=-0.520 total time=  11.3s
[CV 5/5] END estimator=LogisticRegression(random_state=24), n_estimators=10;, score=-0.523 total time=  10.9s
[CV 1/5] END estimator=LogisticRegression(random_state=24), n_estimators=20;, score=-0.525 total time=  20.9s
[CV 2/5] END estimator=LogisticRegression(random_state=24), n_estimators=20;, score=-0.518 total time=  21.3s
[CV 3/5] END estimator=LogisticRegression(random_state=24), n_estimators=20;, score=-0.520 total time=  20.7s
[CV 4/5] END estimator=LogisticRegression(random_state=24),

KeyboardInterrupt: 

In [ ]:
print("Best Score: ",gcv.best_score_)
print("Best Params: ",gcv.best_params_)